<a href="https://colab.research.google.com/github/onenechan/GEE_turtrial/blob/main/GEE_sdf_turtrial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install google earth engine api
!python -c "from oauth2client import crypt"
!pip install earthengine-api

# Authorization
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=DO9_-PJD4V9sfTJUEF3Pvu8d98NSwH05n_oKwOe858g&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7pyT_Sqay5KzKbyfxsDIeG57CqYQ7SgLBgAASp_OebylzynonAM4I

Successfully saved authorization token

In [2]:
from IPython.display import Image
import ee
import ee.mapclient

ee.Initialize()

In [3]:
def TrueColor(data,area,gain):
  """
  TrueColor画像の可視化を行うメソッド。
  image : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  """
  red = data.expression("b('B4')*gain",{"gain":gain})
  green = data.expression("b('B3')*gain",{"gain":gain})
  blue = data.expression("b('B2')*gain",{"gain":gain})
  
  # TrueColorのバンドを重ねたデータを作成する。
  bands = ee.Image(red).addBands(green).addBands(blue)
  image = bands.clip(area)
  image_url = image.getThumbUrl({'min':0,'max':1,'dimensions':'800'})
  return Image(url=image_url)

In [4]:
# Input data
Area1 = ee.Geometry.Rectangle(138.7,35.3,138.8,35.4)
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2019-03-01','2019-03-15').min()

In [5]:
TrueColor(Landsat8_collection,Area1,1)

In [7]:
def IdentifyCloud(data,area):
  """
  雲を判別して可視化するメソッド
  data : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  """
  
  #雲と雪の判別を行うためのバンド6,7の反射率の基準値を設定
  cloud_lim = 0.20
  
  #()?がif,:()?がelse if,:がelse文に対応
  #b()でバンドを選択できる
  #バンド6,7がともに基準値以上であれば雲と判断し、青色にする
  red = data.expression("(b('B6')>lim && b('B7')>lim)? 0"+ ": b('B4')",{"lim":cloud_lim})
  green = data.expression("(b('B6')>lim && b('B7')>lim)? 0" + ": b('B3')",{"lim":cloud_lim})
  blue = data.expression("(b('B6')>lim && b('B7')>lim)? 1"+": b('B2')",{"lim":cloud_lim})
  
  #TrueColorのバンドを重ねたデータを作成
  image = ee.Image(red).addBands(green).addBands(blue)
  
  #データの範囲を選択
  clipped_image = image.clip(area)
  
  #画像のサムネイルの作成と画像の可視化
  image_url = clipped_image.getThumbUrl({'min':0,'max':1,'dimentions':'800'})
  print(image_url)
  return Image(url=image_url)

In [8]:
IdentifyCloud(Landsat8_collection,Area1)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ef27f7b204ab212161ac372337e34132-22d66fc229e3fe31d7c52f498d23892a:getPixels


In [10]:
def NDVI(data,area):
  #ndvi = data.normalizedDifference(['B5','B4'])でも良い
  ndvi = data.expression("(b('B5')-b('B4'))/(b('B5')+b('B4'))")
  
  #雪と雲を検知(簡易版)
  cloud_lim = 0.2
  snow_lim = 0.3
  snow = data.expression("(b('B2')>lim1 && b('B3')>lim1 && b('B4')>lim1 && b('B6')<lim2 && b('B7')<lim2)? 1"+": 0",{"lim1":snow_lim,"lim2":cloud_lim})
  cloud = data.expression("(b('B6')>lim && b('B7')>lim)? 1"+": 0",{"lim":cloud_lim})
 
  #NDVIを緑色、雪を赤色、雲を青色で表現&範囲を選択
  image = ee.Image(snow).addBands(ndvi).addBands(cloud).clip(area)
  
  #NDVIの-1〜0の値を切り捨てて可視化
  image_url = image.getThumbUrl({'min':0,'max':1,'dimentions':1600})
  return Image(url=image_url)

In [16]:
# Input data
Area1 = ee.Geometry.Rectangle(139,35,141,37)
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2018-01-01','2018-04-08').median()

In [18]:
TrueColor(Landsat8_collection,Area1,3)

In [24]:
NDVI(Landsat8_collection,Area1)